In [2]:
import numpy as np
import os
from f_meas_gen import build_measurement_plan

In [3]:
#Carrega Rede
Ybus_File = "Rede_2224bus_GB.txt"
#Ybus_File = "ieee-6-bus.txt"
Ybus = np.loadtxt(Ybus_File, dtype='i', delimiter=',')

In [4]:
def count_branches(Ybus):
    n_branches = 0
    
    for line in Ybus:
        for i in line:
            n_branches += i
    n_branches = int(n_branches/2)
    return n_branches

In [5]:
class Bus_system:
    def __init__(self, Ybus):
        self.Ybus=Ybus
        self.n_bus =  np.size(Ybus,1)
        self.n_branches = count_branches(Ybus)
        self.max_meas = self.n_branches*2 + self.n_bus


In [7]:
#Parametros da Topologia
power_system = Bus_system(Ybus)

In [8]:
measurement_plan = build_measurement_plan(power_system,.8)

UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM


C:\Users\ayres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\linalg\linalg.py:2158: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [9]:
np.size(measurement_plan,0)

6784

In [11]:
np.savetxt('medplan6784medidas.txt',measurement_plan,fmt='%1d')